In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from scipy.signal import convolve

from sklearn.preprocessing import StandardScaler
from gtda.time_series import SingleTakensEmbedding

from gtda.homology import VietorisRipsPersistence, WeightedRipsPersistence

from sklearn.decomposition import PCA

In [ ]:
def diagram_reshape(diagram):
    zero_idx = np.where(diagram[:,2]==0)
    one_idx = np.where(diagram[:,2]==1)
    return diagram[zero_idx], diagram[one_idx]

In [ ]:
def moving_average(ts, kernel_width=10, mode="same"):
    return convolve(ts, np.ones(kernel_width)/kernel_width, mode=mode)

### Data

#### Lynx + hare

In [ ]:
url = "http://people.whitman.edu/~hundledr/courses/M250F03/LynxHare.txt"

In [ ]:
X = pd.read_csv(url, delim_whitespace=True, header=None, index_col=0).to_numpy()
#X = StandardScaler().fit_transform(X)
X.shape

In [ ]:
X_ma = np.zeros_like(X)
X_ma[:,0] = moving_average(X[:,0], kernel_width=12)
X_ma[:,1] = moving_average(X[:,1], kernel_width=12)

In [ ]:
te = SingleTakensEmbedding(parameters_type="fixed", time_delay=2, dimension=2)

In [ ]:
X_detrended = X - X_ma

In [ ]:
X_delay_hare = te.fit_transform(X_detrended[:,0])
X_delay_lynx = te.fit_transform(X_detrended[:,1])

X_delay_hare.shape, X_delay_lynx.shape

In [ ]:
for kernel_width in range(2, 12+2):
    
    X_ma = np.zeros_like(X)
    X_ma[:,0] = moving_average(X[:,0], kernel_width)
    X_ma[:,1] = moving_average(X[:,1], kernel_width)
    
    X_detrended = X - X_ma
    
    X_delay_hare = te.fit_transform(X_detrended[:,0])
    X_delay_lynx = te.fit_transform(X_detrended[:,1])

    fig, ax = plt.subplots(ncols=4, figsize=(16, 4), dpi=200)
    plt.suptitle("Kernel width: {}".format(kernel_width))
    ax[0].plot(X_detrended[:,0], c="b", label="Hare")
    ax[0].plot(X_detrended[:,1], c="r", label="Lynx")
    ax[0].plot(X_ma[:,0], c="b", alpha=0.3, label="Hare MA")
    ax[0].plot(X_ma[:,1], c="r", alpha=0.3, label="Lynx MA")
    ax[0].set_xlabel("Year")
    ax[0].set_ylabel("Population, thousands")
    ax[0].legend()
    ax[1].plot(X_detrended[:,0], X_detrended[:,1], marker="+", c="g")
    ax[1].set_xlabel("Hare")
    ax[1].set_ylabel("Lynx")
    ax[2].plot(X_delay_hare[:,0], X_delay_hare[:,1], c="b", marker="+")
    ax[2].set_xlabel("$x_i$")
    ax[2].set_ylabel("$x_{i+1}$")
    ax[3].plot(X_delay_lynx[:,0], X_delay_lynx[:,1], c="r", marker="+")
    ax[3].set_xlabel("$x_i$")
    ax[3].set_ylabel("$x_{i+1}$")
    plt.tight_layout()
    plt.show()

In [ ]:
vr = VietorisRipsPersistence()
dtm = WeightedRipsPersistence(weight_params={"n_neighbors": 3, "r": 2})

In [ ]:
diagram_vr = diagram_reshape(vr.fit_transform(np.expand_dims(X_delay_hare, 0))[0])
diagram_dtm = diagram_reshape(dtm.fit_transform(np.expand_dims(X_delay_hare, 0))[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(16, 4), dpi=200)

plt.suptitle("Kernel width: {}".format(kernel_width))

ax[0].plot(X_detrended[:,0], c="b", label="Hare")
ax[0].plot(X_detrended[:,1], c="r", label="Lynx")
ax[0].set_xlabel("Year")
ax[0].set_ylabel("Population, thousands")
ax[0].legend()

ax[1].plot(X_delay_hare[:,0], X_delay_hare[:,1], c="r", marker="+")
ax[1].set_xlabel("$x_i$")
ax[1].set_ylabel("$x_{i+1}$")

ax[2].scatter(diagram_vr[0][:,0], diagram_vr[0][:,1], c="r")
ax[2].scatter(diagram_vr[1][:,0], diagram_vr[1][:,1], c="b")
ax[2].set_xlabel("Birth")
ax[2].set_ylabel("Death")

ax[3].scatter(diagram_dtm[0][:,0], diagram_dtm[0][:,1], c="r")
ax[3].scatter(diagram_dtm[1][:,0], diagram_dtm[1][:,1], c="b")
ax[3].set_xlabel("Birth")
ax[3].set_ylabel("Death")

plt.tight_layout()
plt.show()

#### Indian weather

In [ ]:
url = "data/DailyDelhiClimateTrain.csv"

In [ ]:
cols = ["meantemp", "humidity"]

X = pd.read_csv(url, usecols=cols).to_numpy()
X.shape

In [ ]:
X_ma = np.zeros_like(X)
X_ma[:,0] = moving_average(X[:,0], kernel_width=10)
X_ma[:,1] = moving_average(X[:,1], kernel_width=10)

In [ ]:
X_detrended = X

In [ ]:
te = SingleTakensEmbedding(parameters_type="fixed", time_delay=25, dimension=4)

In [ ]:
X_delay_temperature = te.fit_transform(X_ma[:,0])
X_delay_humidity = te.fit_transform(X_ma[:,1])

X_delay_temperature.shape, X_delay_humidity.shape

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(16, 4), dpi=200)
ax[0].plot(X_ma[:,0], c="b")
ax[0].plot(X_ma[:,1], c="r")
ax[1].plot(X_ma[:,0], X[:,1], marker="+", c="g", alpha=0.2)
ax[2].plot(X_delay_temperature[:,0], X_delay_temperature[:,3], c="b", marker="+", alpha=0.2)
ax[3].plot(X_delay_humidity[:,0], X_delay_humidity[:,2], c="r", marker="+", alpha=0.2)
plt.tight_layout()
plt.show()

In [ ]:
vr = VietorisRipsPersistence()
dtm = WeightedRipsPersistence(weight_params={"n_neighbors": 10, "r": .1})

In [ ]:
diagram_vr = diagram_reshape(vr.fit_transform(np.expand_dims(X_delay_humidity, 0))[0])
diagram_dtm = diagram_reshape(dtm.fit_transform(np.expand_dims(X_delay_humidity, 0))[0])

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(16, 4), dpi=200)

plt.suptitle("Kernel width: {}".format(kernel_width))

ax[0].plot(X[:,0], c="b", label="Hare")
ax[0].plot(X[:,1], c="r", label="Lynx")
ax[0].set_xlabel("Year")
ax[0].set_ylabel("Population, thousands")
ax[0].legend()

ax[1].plot(X_delay_humidity[:,0], X_delay_humidity[:,2], c="r", marker="+", alpha=0.2)
ax[1].set_xlabel("$x_i$")
ax[1].set_ylabel("$x_{i+1}$")

ax[2].scatter(diagram_vr[0][:,0], diagram_vr[0][:,1], s=15, c="r")
ax[2].scatter(diagram_vr[1][:,0], diagram_vr[1][:,1], s=15, c="b")
ax[2].set_xlabel("Birth")
ax[2].set_ylabel("Death")

ax[3].scatter(diagram_dtm[0][:,0], diagram_dtm[0][:,1], s=15, c="r")
ax[3].scatter(diagram_dtm[1][:,0], diagram_dtm[1][:,1], s=15, c="b")
ax[3].set_xlabel("Birth")
ax[3].set_ylabel("Death")

plt.tight_layout()
plt.show()

In [ ]:
for kernel_width in range(100, 1000+1, 100):
    
    X_ma = np.zeros_like(X)
    X_ma[:,0] = moving_average(X[:,0], kernel_width)
    X_ma[:,1] = moving_average(X[:,1], kernel_width)
    
    X_detrended = X - X_ma
    
    X_delay_temp = te.fit_transform(X_detrended[:,0])
    X_delay_humidity = te.fit_transform(X_detrended[:,1])

    fig, ax = plt.subplots(ncols=4, figsize=(16, 4), dpi=200)
    plt.suptitle("Kernel width: {}".format(kernel_width))
    ax[0].plot(X_detrended[:,0], c="b", label="Temp")
    ax[0].plot(X_detrended[:,1], c="r", label="Humidity")
    #ax[0].set_xlabel("Year")
    #ax[0].set_ylabel("Population, thousands")
    ax[0].legend()
    ax[1].plot(X_detrended[:,0], X_detrended[:,1], marker="+", c="g")
    ax[1].set_xlabel("Temp")
    ax[1].set_ylabel("Humidity")
    ax[2].plot(X_delay_temp[:,0], X_delay_temp[:,1], c="b", marker="+")
    ax[2].set_xlabel("$x_i$")
    ax[2].set_ylabel("$x_{i+1}$")
    ax[3].plot(X_delay_humidity[:,0], X_delay_humidity[:,1], c="r", marker="+")
    ax[3].set_xlabel("$x_i$")
    ax[3].set_ylabel("$x_{i+1}$")
    plt.tight_layout()
    plt.show()